In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import folium
from folium import Marker, Circle, Choropleth
from folium.plugins import MarkerCluster


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
        #../input/cities/cities.csv
        
data = pd.read_csv("/kaggle/input/cost-of-living-family-of-four-without-rent/Cost_Of_Living_Family.csv", error_bad_lines=False)
data.describe()
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install geopy==1.22.0
import math
import pandas as pd
import geopandas as gpd
import numpy as np
import folium
from folium import Marker
from folium.plugins import MarkerCluster
from geopandas.tools import geocode
result = geocode("The Great Pyramid of Giza", provider="nominatim")
result

In [ ]:
point = result.geometry.iloc[0]
print("Latitude:", point.y)
print("Longitude:", point.x)

In [ ]:
def my_geocoder(row):
    try:
        point = geocode(row, provider='nominatim').geometry.iloc[0]
        return pd.Series({'Latitude': point.y, 'Longitude': point.x, 'geometry': point})
    except:
        return None

data[['Latitude', 'Longitude', 'geometry']] = data.apply(lambda x: my_geocoder(x['Country']), axis=1)

print("{}% of addresses were geocoded!".format(
    (1 - sum(np.isnan(data["Latitude"])) / len(data)) * 100))

# Drop data that were not successfully geocoded
data = data.loc[~np.isnan(data["Latitude"])]
data = gpd.GeoDataFrame(data, geometry=data.geometry)
data.crs = {'init': 'epsg:4326'}
data.head()

In [ ]:
# Create the map
v_map = folium.Map(location=[39.760982,10.00065], tiles='cartodbpositron', zoom_start=2) 

# Add points to the map
mc = MarkerCluster()
for idx, row in data.iterrows(): 
    if not math.isnan(row['Longitude']) and not math.isnan(row['Latitude']):
        mc.add_child(Marker(location=[row['Latitude'], row['Longitude']],
                            tooltip=str(row['Family_Cost_Of_Living'])+" USD $"))
v_map.add_child(mc)

#Style map
folium.TileLayer('stamentoner').add_to(v_map)

# Display the map
v_map